In [ ]:
import os 
import time
import pandas as pd


import keras 
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Input

from keras import layers
from keras.layers import Conv2DTranspose
from tensorflow.keras import backend as K
import numpy as np
import librosa
import matplotlib.pyplot as plt
import librosa.display

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score


# Electrical engine sound anomaly detection

## Model setup ( CNN AutoEncoder ) 

In [ ]:
class ConvAutoencoder:
    @staticmethod
    def build(width, height, depth, filters=(32, 64), latentDim=50):
        # initialize the input shape to be "channels last" along with
        # the channels dimension itself
        # channels dimension itself
        inputShape = (height, width, depth)
        chanDim = -1
        # define the input to the encoder
        inputs = Input(shape=inputShape)
        x = inputs
        # loop over the number of filters
        for f in filters:
        # apply a CONV => RELU => BN operation
            x = Conv2D(f, (3, 3), strides=2, padding="same")(x)
            x = LeakyReLU(alpha=0.2)(x)
            x = BatchNormalization(axis=chanDim)(x)
        # flatten the network and then construct our latent vector
        volumeSize = K.int_shape(x)
        x = Flatten()(x)
        latent = Dense(latentDim)(x)
        # build the encoder model
        encoder = Model(inputs, latent, name="encoder")
        # start building the decoder model which will accept the
        # output of the encoder as its inputs
        latentInputs = Input(shape=(latentDim,))
        x = Dense(np.prod(volumeSize[1:]))(latentInputs)
        x = Reshape((volumeSize[1], volumeSize[2], volumeSize[3]))(x)
        # loop over our number of filters again, but this time in
        # reverse order
        for f in filters[::-1]:
            # apply a CONV_TRANSPOSE => RELU => BN operation
            x = Conv2DTranspose(f, (3, 3), strides=2,
                padding="same")(x)
            x = LeakyReLU(alpha=0.2)(x)
            x = BatchNormalization(axis=chanDim)(x)
        # apply a single CONV_TRANSPOSE layer used to recover the
        # original depth of the image
        x = Conv2DTranspose(depth, (3, 3), padding="same")(x)
        outputs = Activation("sigmoid")(x)
        # build the decoder model
        decoder = Model(latentInputs, outputs, name="decoder")
        # our autoencoder is the encoder + decoder
        autoencoder = Model(inputs, decoder(encoder(inputs)),
        name="autoencoder")
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
        autoencoder.compile(loss="mse", optimizer=optimizer)
        return autoencoder

## Data formating

In [ ]:
def build_spectrograms(input_folder,saving_folder):
    files = librosa.util.find_files(input_folder, ext=['wav']) 
    files = np.asarray(files)
    for y in files:  
        data = librosa.load(y, sr = 16000,mono = True)   
        data = data[0]     
        S_left = librosa.stft(data, center=True)
        fig, ax = plt.subplots()
        img = librosa.display.specshow(librosa.amplitude_to_db(S_left,
                                                               ref=np.max),
                                       y_axis='log', x_axis='time', ax=ax)
        ax.set_title('Power spectrogram')
        fig.colorbar(img, ax=ax, format="%+2.0f dB")
        file_name = y.split('/')[-1].split('.')[0]
        plt.savefig(f"{saving_folder}/{file_name}.png")
   

In [ ]:
# load and transform sound to spectrogram
folders = [("./IDMT-ISA-ELECTRIC-ENGINE/train_cut/engine1_good","./Spectrograms/train_cut/engine1_good"),
           ("./IDMT-ISA-ELECTRIC-ENGINE/train_cut/engine2_broken","./Spectrograms/train_cut/engine2_broken"),
           ("./IDMT-ISA-ELECTRIC-ENGINE/train_cut/engine3_heavyload","./Spectrograms/train_cut/engine3_heavyload"),
           ("./IDMT-ISA-ELECTRIC-ENGINE/test_cut/engine1_good","./Spectrograms/test_cut/engine1_good"),
           ("./IDMT-ISA-ELECTRIC-ENGINE/test_cut/engine2_broken","./Spectrograms/test_cut/engine2_broken"),
           ("./IDMT-ISA-ELECTRIC-ENGINE/test_cut/engine3_heavyload","./Spectrograms/test_cut/engine3_heavyload")]

for input_folder,saving_folder in folders:
    build_spectrograms(input_folder,saving_folder)

## Spectrograms loading

In [ ]:
# Custom loading of training data
train_folder = "Spectrograms/train_cut/"
train_images = []
ano_images = [] 
class_data = ["engine1_good","engine2_broken","engine3_heavyload"]
for label in class_data:
    for i,img in enumerate(os.listdir(f"{train_folder}/{label}/")):
        if 'png' in img:
            image = tf.keras.utils.load_img(f"{train_folder}/{label}/{img}",color_mode = "grayscale",target_size=(200, 200))
            input_arr = tf.keras.utils.img_to_array(image)/255.0
            if label in ["engine1_good","engine3_heavyload"]:    
                train_images.append(input_arr)
            else:
                ano_images.append(input_arr)


In [ ]:
X_train0_CAE = np.array(train_images) # Training data without anomalies
X_train1_CAE = np.array(ano_images) # Training data with anomalies

In [ ]:
# Custom loading of test data
test_folder = "Spectrograms/test_cut"
test_images0 = []
test_images1 = [] 
class_data = ["engine1_good","engine2_broken","engine3_heavyload"]
for label in class_data:
    for i,img in enumerate(os.listdir(f"{test_folder}/{label}/")):
        if 'png' in img:
            image = tf.keras.utils.load_img(f"{test_folder}/{label}/{img}",color_mode = "grayscale",target_size=(200, 200))
            input_arr = tf.keras.utils.img_to_array(image)/255.0
            if label in ["engine1_good","engine3_heavyload"]:    
                test_images0.append(input_arr)
            else:
                test_images1.append(input_arr)

In [ ]:
X_test0CAE = np.array(test_images0) # Test data without anomalies
X_test1CAE = np.array(test_images1) # Test data with anomalies

## Model learning

In [ ]:
CAE = ConvAutoencoder.build(200, 200, 1) # Model architecture 

In [ ]:
start = time.time()
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)
CAE.fit(X_train0_CAE,X_train0_CAE, epochs=50,batch_size = 10, verbose = True, callbacks= [callback])
print(f"model trained in {(time.time()-start)/60} minutes")

In [ ]:
CAE.save('CAE200_200.h5')

## Prediction

In [ ]:
prediction_test0CAE = CAE.predict(X_test0CAE)
prediction_test1CAE = CAE.predict(X_test1CAE)

In [ ]:
r_e_test0CAE = np.sum(np.sum((X_test0CAE-prediction_test0CAE) ** 2, axis=1),axis=1)
r_e_test1CAE = np.sum(np.sum((X_test1CAE-prediction_test1CAE) ** 2, axis=1),axis=1)

In [ ]:
re_test = np.concatenate((r_e_test0CAE, r_e_test1CAE))

In [ ]:
threshold_t_CAE = np.quantile(re_test,0.7)

In [ ]:
decision0CAE = -r_e_test0CAE + threshold_t_CAE
decision1CAE = -r_e_test1CAE + threshold_t_CAE
df0CAE = pd.DataFrame(decision0CAE, columns=['v'])
df1CAE = pd.DataFrame(decision1CAE, columns=['v'])
decision1CAE = -r_e_test1CAE

In [ ]:
# Computing f1 score of majority class
recall = len(df1CAE[df1CAE.v<0])/len(df1CAE)
precision = len(df1CAE[df1CAE.v<0])/(len(df1CAE[df1CAE.v<0])+len(df0CAE[df0CAE.v<0]))
f1 = 2*precision*recall/(precision+recall)

In [ ]:
# Computing f1 score of majority class
recall_maj = len(df0CAE[df0CAE.v>0])/len(df0CAE)
precision_maj = len(df0CAE[df0CAE.v>0])/(len(df1CAE[df1CAE.v>0])+len(df0CAE[df0CAE.v>0]))
f1_maj = 2*precision_maj*recall_maj/(precision_maj+recall_maj)

In [ ]:
df1CAE.describe()

### Test score

In [ ]:
print(f"Test scores:")
print(f"minority class F1 score {f1}")
print(f"majority class F1 score {f1_maj}")
print(f"average F1 score {(f1+f1_maj)/2}")